*If running in Colab run this first to install ACN-Portal.*

In [ ]:
import subprocess
import sys

if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    subprocess.check_call([sys.executable, "-m", "pip", "install", "acnportal"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "git+https://github.com/caltech-netlab/adacharge"])

# Baseline Algorithm Evaluation
In this experiment we compare the performance of the Round Robin, First-Come First-Served, Earliest Deadline First, and Least Laxity First algorithms. To understand how these algorithms cope with constrained infrastructure, we limit the capacity of the transformer feeding the charging network. We then compare what percentage of energy demands each algorithm is able to meet. We also consider the current unbalance caused by each algorithm to help understand why certain algorithms are able to deliver more or less energy at a given infrastrucutre capacity.

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import pytz
import numpy as np
import pandas as pd
from collections import defaultdict
from pprint import pprint
from copy import deepcopy
import json
import os
import time
import gzip
import random

from acnportal import acnsim
from acnportal.acnsim import analysis,network
from acnportal.signals.tariffs import TimeOfUseTariff
from acnportal import algorithms
import adacharge

import importlib
importlib.reload(adacharge)

In [ ]:
matplotlib.rcParams.update({'font.size': 12})

## Experiment Parameters


In [ ]:
# Timezone of the ACN we are using.
timezone = pytz.timezone('America/Los_Angeles')

# Start and End times are used when collecting data.
start = timezone.localize(datetime(2018, 12, 14))

# Use only one week for quicker results.
end = timezone.localize(datetime(2018, 12, 15))

# How long each time discrete time interval in the simulation should be.
period = 5  # minutes

# Voltage of the network.
voltage = 208  # volts

# Default maximum charging rate for each EV battery.
default_battery_power = 6.6  # kW

# Site info
site = 'jpl'
station_ids = acnsim.sites.jpl_acn().station_ids

In [ ]:
# Get Events Via the API
API_KEY = 'DEMO_TOKEN'
events = acnsim.acndata_events.generate_events(API_KEY, site, start, end,
                                               period, voltage,
                                               default_battery_power)

In [ ]:
# Save Events
start_iso = start.strftime("%Y%m%dT%H%M%S")
end_iso = end.strftime("%Y%m%dT%H%M%S")
name = f"/content/sample_data/{site}_{start_iso}_{end_iso}_{period}m_{voltage}V_{default_battery_power}kW.json.gz"
if not os.path.exists(name):
    data = events.to_json()
    with gzip.GzipFile(name, 'w') as fout:
        fout.write(json.dumps(data).encode('utf-8'))

In [ ]:
# # Load Events
# start_iso = start.strftime("%Y%m%dT%H%M%S")
# end_iso = end.strftime("%Y%m%dT%H%M%S")
# name = f"/content/sample_data/{site}_{start_iso}_{end_iso}_{period}m_{voltage}V_{default_battery_power}kW.json.gz"
# if os.path.exists(name):
#     with gzip.GzipFile(name, 'r') as fin:
#         data = json.loads(fin.read().decode('utf-8'))
#         events = acnsim.EventQueue.from_json(data)
# else:
#     print("No cached events with the given parameters.")

In [ ]:
# Load Events
start_iso = start.strftime("%Y%m%dT%H%M%S")
end_iso = end.strftime("%Y%m%dT%H%M%S")
name = f"/content/10_12_jpl_user.json.gz"
if os.path.exists(name):
    with gzip.GzipFile(name, 'r') as fin:
        data = json.loads(fin.read().decode('utf-8'))
        events_real = acnsim.EventQueue.from_json(data)
else:
    print("No cached events with the given parameters.")

In [ ]:
# Load Events
start_iso = start.strftime("%Y%m%dT%H%M%S")
end_iso = end.strftime("%Y%m%dT%H%M%S")
name = f"/content/14_12_jpl_predicted.json.gz"
if os.path.exists(name):
    with gzip.GzipFile(name, 'r') as fin:
        data = json.loads(fin.read().decode('utf-8'))
        events_predicted = acnsim.EventQueue.from_json(data)
else:
    print("No cached events with the given parameters.")

In [ ]:
print(network.ChargingNetwork.constraints_as_df)

## Algorithms
We consider four algorithms: Round Robin, First-Come First-Served, Earliest Deadline First, and Least Laxity First.

In [ ]:
def days_remaining_scale_demand_charge(rates, infrastructure, interface,
                                       baseline_peak=0, **kwargs):
    day_index = interface.current_time // ((60 / interface.period) * 24)
    days_in_month = 30
    day_index = min(day_index, days_in_month - 1)
    scale = 1 / (days_in_month - day_index)
    dc = adacharge.demand_charge(rates, infrastructure, interface, baseline_peak, **kwargs)
    return scale * dc

In [ ]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['FCFS'] = algorithms.SortedSchedulingAlgo(algorithms.first_come_first_served)
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)
sch['EDF'] = algorithms.SortedSchedulingAlgo(algorithms.earliest_deadline_first)
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served)

# quick_charge_obj = [adacharge.ObjectiveComponent(adacharge.quick_charge),
#                     adacharge.ObjectiveComponent(adacharge.equal_share, 1e-12)]
#sch['MPC_QC'] = adacharge.AdaptiveSchedulingAlgorithm(quick_charge_obj, solver="ECOS")
#sch['Offline_QC'] = adacharge.AdaptiveChargingAlgorithmOffline(quick_charge_obj, solver="ECOS")

cost_min_obj = [adacharge.ObjectiveComponent(adacharge.total_energy,1000),
                #adacharge.ObjectiveComponent(days_remaining_scale_demand_charge),
                #adacharge.ObjectiveComponent(adacharge.quick_charge, 1e-4),
                #adacharge.ObjectiveComponent(adacharge.equal_share, 1e-12),
                adacharge.ObjectiveComponent(adacharge.load_flattening)
               ]

sch['MPC_CM'] = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="ECOS")

# cost_min_obj_off = [
#                     adacharge.ObjectiveComponent(adacharge.total_energy, 1000),
#                     adacharge.ObjectiveComponent(adacharge.tou_energy_cost),
#                     adacharge.ObjectiveComponent(adacharge.demand_charge),
#                    ]
#sch['Offline_CM'] = adacharge.AdaptiveChargingAlgorithmOffline(cost_min_obj_off, solver="MOSEK")

## Experiment

Load results from the data directory.

In [ ]:
# sims = dict()
# results_dir = "results/sims"
# if os.path.exists(results_dir):
#     for filename in os.listdir(results_dir):
#         try:
#             split_name = filename.split(".")
#             if "gz" == split_name[-1]:
#                 network_type, alg_name, cap = split_name[0].split("-")
#                 path = os.path.join(results_dir, filename)
#                 with gzip.GzipFile(path, 'r') as fin:
#                     data = json.loads(fin.read().decode('utf-8'))
#                     sims[network_type, alg_name, int(cap)] = acnsim.Simulator.from_json(data)
#         except ValueError:
#             pass

### Run Experiment from Scratch

To run the experiment we vary the capacity of the transformer which feeds the Caltech charging network from 5 kW to 150 kW. This allows us to see how each algorithm copes with various levels of infrastructure constraints.

In [ ]:
sims = dict()

Transformer 1st floor - default is 45 so vary from 10 to 45 then keep at 45
Tansformeer of 3rd and 4th floor - default is 150 so vary from 10 to 150
Constrained Infrastructure

In [ ]:
def run_experiment(network_type, alg_name):
    if network_type == "single_phase":
        cn = acnsim.sites.simple_acn(station_ids, voltage=voltage)
    else:
        cn = acnsim.sites.jpl_acn(basic_evse=True, voltage=voltage)
    alg = deepcopy(sch[alg_name])
    experiment_events = deepcopy(events_real)
    signals = {'tariff': TimeOfUseTariff("sce_tou_ev_4_march_2019")}
    sim = acnsim.Simulator(cn, alg, experiment_events, start, period=period, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "Offline_CM" or alg_name == "Offline_QC":
        alg.register_events(experiment_events)
        alg.solve()
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")
    return sim

In [ ]:
def run_experiment_predicted(network_type, alg_name):
    if network_type == "single_phase":
        cn = acnsim.sites.simple_acn(station_ids, voltage=voltage)
    else:
        cn = acnsim.sites.jpl_acn(basic_evse=True, voltage=voltage)
    alg = deepcopy(sch[alg_name])
    experiment_events = deepcopy(events_predicted)
    signals = {'tariff': TimeOfUseTariff("sce_tou_ev_4_march_2019")}
    sim = acnsim.Simulator(cn, alg, experiment_events, start, period=period, signals=signals, verbose=False)
    #sim = acnsim.Simulator(cn, alg, experiment_events, start, period=period, verbose=False)
    print("Running...")
    start_simulation = time.time()
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")
    return sim

In [ ]:
# We expect Unctrl to overload the system, so we will surpress warnings.
import warnings
warnings.simplefilter("ignore")

#capacities = list(range(10, 126, 5))
alg_names = ["Unctrl","FCFS", "EDF", "LLF", "MPC_CM"]
#alg_names = ["Unctrl", "MPC_CM"]

#for network_type in ["single_phase", "three_phase"]:
for network_type in ["three_phase"]:
    for alg_name in alg_names:
      config = (network_type, alg_name)
      print(config)
      print(network.ChargingNetwork.constraints_as_df)
      if config not in sims:
          sims[config] = run_experiment(*config)

In [ ]:
# We expect Unctrl to overload the system, so we will surpress warnings.
import warnings
warnings.simplefilter("ignore")

#capacities = list(range(10, 126, 5))
alg_names = ["Unctrl","RR", "FCFS", "EDF", "LLF", "MPC_CM"]
#alg_names = ["Unctrl", "MPC_CM"]

#for network_type in ["single_phase", "three_phase"]:
for network_type in ["three_phase"]:
    for alg_name in alg_names:
      config = (network_type, alg_name)
      print(config)
      if config not in sims:
          sims[config] = run_experiment_predicted(*config)

In [ ]:
if not os.path.exists("results/sims"):
    os.makedirs("results/sims")

for config, sim in sims.items():
    name = "results/sims/{0}-{1}.json.gz".format(*config)
    if not os.path.exists(name):
        data = sim.to_json()
        with gzip.GzipFile(name, 'w') as fout:
            fout.write(json.dumps(data).encode('utf-8'))

## Results
We can then analyze the results of the experiment. We consider three metrics.

1.   Proportion of total energy requested which is delivered by each algorithm.
2.   Maximum infrastructure utilization, defined as the maximum instantaneous power draw over the capacity of the system.
3.   Average current unbalance which measures how well each algorithm is able to balance between phases, a key factor in maximally utilizing infrastructure capacity.

In [ ]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": config[0],
        "Algorithm": config[1],
        #"Capacity (kW)": config[2],
        "Aggregate Power (kW)": analysis.aggregate_power(sim),
        "Peak (kW)": np.max(analysis.aggregate_power(sim)),
        "Demand Charge": analysis.demand_charge(sim),
        "Energy Cost" : analysis.energy_cost(sim)
    }
    return metrics

In [ ]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items())

In [ ]:
sim_list=[]
for config, sim in sims.items():
  sim_list.append(sim)

In [ ]:
len(analysis.aggregate_power(sim_list[2]))

In [ ]:
three_phase_real_MPC=[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.66195652,  1.71155448,
        1.71155448,  1.71155448,  1.71155448,  1.71155448,  1.71155448,
        1.71155448,  1.71155448,  2.77559301,  2.77559301,  2.77559301,
        2.77559301,  2.77559301,  3.17922321,  5.85943256,  5.85943256,
        5.85943256,  5.85943256,  5.85943256,  5.85943256,  7.74525411,
        7.74525411,  8.22144559,  9.21548274,  9.21548274,  9.21548274,
       11.25612665, 11.25612665, 11.25612665, 11.25612665, 11.25612665,
       11.25612665, 11.25612665, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 11.81517498, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 11.81517498, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 11.81517498, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 11.81517498, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 11.81517498, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 11.81517498, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 11.81517498, 11.81517498, 11.81517498, 11.81517498,
       11.81517498, 14.34011475, 14.34011475, 14.34011475, 14.34011475,
       14.34011475, 14.34011475, 14.34011475, 14.34011475, 14.34011475,
       14.34011475, 14.34011475, 14.34011475, 14.34011475, 14.34011475,
       14.34011475, 14.34011475, 14.34011475, 14.34011475, 14.34011475,
       14.34011475, 13.21441918, 12.37656931, 12.37656931, 12.37656931,
       12.37656931, 12.37656931, 10.90207126, 10.4679885 , 10.4679885 ,
       10.4679885 ,  9.93367194,  9.40183667,  9.40183667,  9.40183667,
        9.40183667,  8.9083365 ,  8.74180715,  8.74180715,  8.74180715,
        8.33067027,  7.58066222,  6.85680619,  6.85680619,  6.69205251,
        6.3932711 ,  6.3932711 ,  6.3932711 ,  6.3932711 ,  6.3932711 ,
        6.3932711 ,  6.3932711 ,  6.3932711 ,  6.3932711 ,  6.3932711 ,
        6.3932711 ,  6.3932711 ,  6.3932711 ,  6.3932711 ,  6.3932711 ,
        6.3932711 ,  5.58384933,  5.27762965,  5.27762965,  5.27762965,
        5.27762965,  5.6324412 ,  5.6324412 ,  5.6324412 ,  5.6324412 ,
        5.6324412 ,  5.6324412 ,  5.6324412 ,  5.6324412 ,  5.6324412 ,
        5.6324412 ,  5.6324412 ,  5.6324412 ,  5.6324412 ,  5.6324412 ,
        5.6324412 ,  5.6324412 ,  5.6324412 ,  5.6324412 ,  5.6324412 ,
        5.6324412 ,  5.6324412 ,  5.63244119,  5.63244119,  5.6324412 ,
        5.6324412 ,  5.6324412 ,  5.6324412 ,  5.63244119,  5.63244119,
        5.6324412 ,  5.63244119,  5.48449489,  5.02841435,  5.02841435,
        1.7244833 ,  1.7244833 ,  2.25325043,  2.25325043,  2.25325043,
        2.25325043,  2.25325043,  2.25325043,  2.25325043,  2.25325043,
        2.25325043,  2.25325043,  2.40984934,  2.40984934,  4.25096516,
        4.25096516,  4.25096516,  4.25096516,  4.25096516,  4.25096516,
        4.25096516,  4.25096516,  4.25096516,  4.25096516,  4.25096516,
        4.25096516,  4.25096516,  4.25096516,  4.25096516,  4.25096516,
        4.25096516,  4.25096516,  4.25096516,  2.88025391,  2.18870102,
        2.18870102,  2.48838986,  2.48838986,  6.60000011,  6.60000011,
        6.60000015,  6.60000015, 10.85431105, 10.85431105, 10.85431105,
       10.85431105, 10.85431105, 10.10631067,  8.83142567,  5.3787109 ,
        5.3787109 ,  5.3787109 ,  5.3787109 ,  5.3787109 ,  5.24684353,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta

# Example data
three_phase_predicted_uncontrolled = analysis.aggregate_power(sim_list[2])
three_phase_predicted_MPC = analysis.aggregate_power(sim_list[3])

# Time labels
start_time = datetime(2018, 12, 13)
time_labels = [start_time + timedelta(hours=i) for i in np.linspace(0, 24, 278)]

time_labels[-1] = start_time + timedelta(days=1)

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(time_labels, three_phase_predicted_uncontrolled, label='Uncontrolled Charging')
plt.plot(time_labels, three_phase_predicted_MPC, label='Optimized Charging - Predicted Inputs')
plt.plot(time_labels, np.array(three_phase_real_MPC[0:278]), label='Optimized Charging - User Inputs')

plt.xlabel('Time')
plt.ylabel('Power (kW)')


# Formatting the x-axis to show time in HH:MM format
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Interval can be adjusted for better visibility

# Rotate date labels for better visibility
plt.gcf().autofmt_xdate()

plt.legend()
plt.show()


#Result

In [ ]:
metrics.groupby(["Algorithm", "Network Type"])["Demand Charge"].describe() 

In [ ]:
metrics.groupby(["Algorithm", "Network Type"])["Peak (kW)"].describe() 

In [ ]:
metrics.groupby(["Algorithm", "Network Type"])["Energy Delivered"].describe() 

In [ ]:
metrics.groupby(["Algorithm", "Network Type"])["Energy Cost"].describe()